<a href="https://colab.research.google.com/github/fsicardir/tp2_modelos/blob/main/tp1_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
import requests
import random
from google.colab import files

In [2]:
def add_incompat(incompatibilities, name1, name2):
  incomp = incompatibilities.get(name1, set()) | {name2}
  incompatibilities[name1] = incomp

def add_time(times, name, time):
  times[name] = int(time)

In [3]:
def parse_file(lines):
  incompatibilities = {}
  times = {}

  for line in lines:
    fields = line.split()
    if fields[0] == 'e':
      add_incompat(incompatibilities, fields[1], fields[2])
      add_incompat(incompatibilities, fields[2], fields[1])
    if fields[0] == 'n':
      add_time(times, fields[1], fields[2])

  return times, incompatibilities
    

In [4]:

class Cloth:
  def __init__(self, name, time, incompatibles):
    self.name = name
    self.time = time
    self.incompatibles = incompatibles

  def __str__(self):
    return f'Cloth({self.name}, {self.time}, {self.incompatibles})'

  def __repr__(self):
    return self.__str__()


In [5]:
def create_clothes(times, incompats):
  clothes = []
  for name, time in times.items():
    clothes.append(Cloth(name, time, incompats.get(name, set())))
  return clothes

In [6]:
def is_compatible(cloth, batch):
  for c in batch:
    if c.name in cloth.incompatibles:
      return False
  return True

def _get_compatible_batch(cloth, batches, non_compatible):
  if len(batches) == 0:
    for elem in non_compatible:
      heapq.heappush(batches, elem)
    return 0, 0, set()
  order, batch_time, batch = heapq.heappop(batches)
  if is_compatible(cloth, batch):
    for elem in non_compatible:
      heapq.heappush(batches, elem)
    return order, batch_time, batch
  non_compatible.append((order, batch_time, batch))
  return _get_compatible_batch(cloth, batches, non_compatible)

def get_compatible_batch(cloth, batches):
  return _get_compatible_batch(cloth, batches, [])

In [7]:
def process(clothes):
  batches = []
  for cloth in clothes:
    order, batch_time, batch = get_compatible_batch(cloth, batches)
    if cloth.time > batch_time:
      batch_time = cloth.time
    batch.add(cloth)
    heapq.heappush(batches, (-batch_time, batch_time, batch))
  return batches
    

In [8]:
lines = requests.get("https://modelosuno.okapii.com/Problemas/segundo_problema.txt").text.split('\n')


In [9]:
lines = [line.strip() for line in lines][:-1]
len(lines)

19490

In [10]:

times, incompats = parse_file(lines)

clothes = create_clothes(times, incompats)

In [11]:
batches = process(clothes)

print(batches)

[(-20, 20, {Cloth(295, 11, {'250', '292', '364', '368', '15', '249', '73', '238', '115', '335', '129', '294', '248', '367', '127', '293', '16', '332', '352', '128', '240', '245', '356', '48', '369', '385', '159', '337', '372', '237', '32', '212', '50', '300', '183', '252', '378', '125', '351', '296', '178', '79', '74', '358', '89', '87', '76', '31', '161', '170', '184', '231', '214', '266', '200', '371', '347', '30', '330', '339', '123', '81', '83', '305', '254', '181', '131', '121', '370', '215', '336', '154', '244', '298', '239', '232', '157', '384', '217', '247', '221', '297', '114', '220', '333', '349', '75', '350', '28', '301', '78', '191', '274', '201', '72', '334', '17', '374', '361', '21', '12', '71', '218', '375', '70', '366', '130', '19', '255', '365', '39', '329', '29', '299', '167', '253', '306', '120', '308', '348', '251', '198', '216', '189', '166'}), Cloth(233, 12, {'286', '276', '364', '67', '56', '192', '249', '84', '41', '126', '368', '73', '115', '312', '34', '321', 

In [12]:
def results(batches):
  acc = 0
  for order, time, clothes in batches:
    acc += time
  return len(batches), acc

In [13]:
num, total_time = results(batches)
print(f'number of batches: {num}')
print(f'total time: {total_time}')

number of batches: 43
total time: 737


In [14]:
min_time = float('Inf')
best_order = []
for i in range(1000000):
  s = random.sample(clothes, len(clothes))
  b = process(s)
  num, total_time = results(b)
  if total_time < min_time:
    min_time = total_time
    best_order = b

print(f'number of batches: {len(best_order)}')
print(f'total time: {min_time}')

number of batches: 35
total time: 579


In [15]:
def write_solution(batches, filename):
  with open(filename, 'w') as file:
    i = 1
    for batch in batches:
      for cloth in batch[-1]:
        file.write(f'{cloth.name} {i}\n')
      i += 1


In [16]:
filename = '/entrega_1.txt'
write_solution(best_order, filename)

In [17]:
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>